# モンテカルロ法

乱数を用いた試行を繰り返すことによって近似的な答えを求める手法を総称してモンテカルロ法（Monte Carlo method）と呼ぶ。

## モンテカルロ法による円周率の計算

ここでは、次のような方法によって、円周率の近似値を計算する。

![モンテカルロ法による円周率の計算](https://tueda.github.io/PS2020SS/notebooks/images/mc_pi.png)
図1. 半径1の四分円と外接する正方形

図1 (a) のような、一辺の長さが 1 の正方形と、その左下隅を中心とする半径 1 の四分円を考える。この正方形の中に繰り返しランダムに点を発生させよう（図1 (b)）。するとランダムに発生させた点が四分円の中に入る確率は四分円の面積に比例することから、

$$
\frac{\text{（四分円の面積）}}{\text{（正方形の面積）}}
\simeq
\frac{\text{（四分円の中に入った点の数）}}{\text{（正方形の中で発生させた点の数）}} ,
\tag{1}
$$

という関係が近似的に成り立つであろう。右辺については、例えば、実際に正方形と四分円を紙に描き、それを壁に貼ってダーツをランダムに当てたりすれば[<sup id="cite_ref-1">[1]</sup>](#cite_note-1)、実験的に測定ができる。四分円の面積は $\pi/4$ で、正方形の面積は $1$ だから、式 (1) の右辺の実験値に $4$ を掛けると円周率 $\pi$ が求まる。大数の法則により、試行回数が多ければ多いほど、近似の精度もよくなることが期待される。

反復処理は計算機の得意とするところなので、式 (1) の右辺を計算機による乱数を使った数値実験によって求めよう（つまりモンテカルロ法である）。一様乱数を 2 回発生することによって、正方形の中のランダムな点 $(x_i, y_i)$ を 1 つ決めることができる（[random 関数](https://docs.python.org/ja/3/library/random.html#random.random)を用いて $0 \le x_i < 1$、$0 \le y_i < 1$ のように発生させる）。この点が四分円の中にあるかどうかは、

$$
\begin{cases}
x^2 + y^2 < 1 , & \text{円の内部} , \\
x^2 + y^2 > 1 , & \text{円の外部} ,
\end{cases}
\tag{2}
$$

のようにして判断することができる[<sup id="cite_ref-2">[2]</sup>](#cite_note-2)。この試行を繰り返し、点が円の内部に入った回数を数えておけば、式 (1) の右辺が数値実験によって求まる。

乱数を用いた数値実験なので、結果は毎回少しずつ違うはずである。いま行っているモンテカルロ法の統計誤差を考えると、試行回数（サンプルサイズ） $n$ に対して、結果に対する相対誤差は $1/\sqrt{n}$ に比例する[<sup id="cite_ref-3">[3]</sup>](#cite_note-3)。つまり、近似の精度を一桁上げたければ、サンプルサイズを 100 倍にする必要がある。モンテカルロ法を使った数値実験では、結果とともに、その誤差の範囲を把握することが重要である[<sup id="cite_ref-4">[4]</sup>](#cite_note-4)。

以上のことを踏まえて、円周率の近似値をモンテカルロ法で求めるプログラムは次のように書ける。相対誤差は $1/\sqrt{n}$ とし、それを結果に掛けることにより（絶対）誤差を算出している。

In [ ]:
import random

n = 10 ** 2  # サンプルサイズ

count = 0  # 中に入った回数を初期化

# 正方形内に繰り返しランダムな点を発生させて、円の中に入った回数を数える。
for _ in range(n):
    x = random.random()
    y = random.random()

    if x ** 2 + y ** 2 < 1:
        count += 1

# 結果と統計誤差
result = 4 * count / n
error = result / n ** 0.5

print(f'試行回数: {n}')
print(f'入った数: {count}')
print(f'結果: {result}')
print(f'誤差: {error}')

## 練習問題

(1) 上のプログラムで、サンプルサイズを変えながら何回か計算を行ってみて、結果と誤差がどのようになるか調べてみよう。また、皆さんは円周率の値を知っている。結果が誤差の範囲内で合っているのか確かめてみよう[<sup id="cite_ref-5">[5]</sup>](#cite_note-5)。

## 可視化する

せっかくなので、直感的理解を助けるために、上のモンテカロ法の数値実験を可視化してみよう（実用性はないが、教育的ではあるだろう）。つまり、ランダムに発生させた点を散布図としてグラフにしてみる。円の内部に入った点は青、円の外部の点は赤で表すとしよう。次のプログラムでは散布図を [Axes オブジェクト](https://matplotlib.org/api/axes_api.html#the-axes-class)の [scatter メソッド](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.scatter.html#matplotlib-axes-axes-scatter) を使って描画し、また、[add_patch メソッド](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.add_patch.html#matplotlib.axes.Axes.add_patch)を使って円を描画している。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

n = 100  # サンプルサイズ

# グラフ用のデータ
xpoints = []
ypoints = []
colors = []

count = 0  # 中に入った回数を初期化

# 正方形内に繰り返しランダムな点を発生させて、円の中に入った回数を数える。
for _ in range(n):
    x = random.random()
    y = random.random()

    if x ** 2 + y ** 2 < 1:
        count += 1
        c = 'blue'  # 内部は青い点
    else:
        c = 'red'   # 外部は赤い点
    
    # グラフ用
    xpoints.append(x)
    ypoints.append(y)
    colors.append(c)

# 結果と統計誤差
result = 4 * count / n
error = result / n ** 0.5

print(f'試行回数: {n}')
print(f'入った数: {count}')
print(f'結果: {result}')
print(f'誤差: {error}')

# グラフを作る
fig, ax = plt.subplots()
ax.scatter(xpoints, ypoints, s=5, c=colors)  # size と color を指定
ax.add_patch(plt.Circle((0, 0), radius=1, fc="none", ec="green", linewidth=2))  # 円を描く
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.grid()
plt.show()

## 球の体積

![三次元球](https://tueda.github.io/PS2020SS/notebooks/images/sphere.png)
図2. 半径 1 の球

上では円周率の近似値を得るために、正方形と四分円の面積の比をモンテカルロ法により求めていた。同様にして、半径 1 の球の体積を求めてみよう。ランダムな点を一辺の長さが 1 の立方体（$0 \le x < 1$、$0 \le y < 1$、$0 \le z < 1$）に発生させ、原点からの距離の二乗が 1 より小さいかどうかで球の内部か外部を判定する。立方体に入っている球の部分は全体の $1/8$ であるから、得られた結果に $8$ を掛ければ球全体の体積が求まる。

In [ ]:
import random

n = 10 ** 6  # サンプルサイズ

count = 0  # 中に入った回数を初期化

# 立方体内に繰り返しランダムな点を発生させて、球の中に入った回数を数える。
for _ in range(n):
    x = random.random()
    y = random.random()
    z = random.random()

    if x ** 2 + y ** 2 + z ** 2 < 1:
        count += 1

# 結果と統計誤差
result = 8 * count / n
error = result / n ** 0.5

print(f'試行回数: {n}')
print(f'入った数: {count}')
print(f'結果: {result}')
print(f'誤差: {error}')

## 演習課題

(1) 曲線 $y = \sqrt{x}$ の $0 \le x \le 1$ の部分と$x$ 軸に囲まれた領域の面積をモンテカルロ法にて求め、結果を誤差とともに表示せよ。相対誤差がおおよそ 1% 以下になるようにサンプルサイズを取れ。

In [ ]:
# 課題解答9.1  <-- 提出する際に、この行を必ず含めること。

import random




(2) 次の不等式で与えられる領域の面積をモンテカルロ法にて求め、結果を誤差とともに表示せよ。相対誤差がおおよそ 1% 以下になるようにサンプルサイズを取れ。

$$
(x^2 + y^2 - 0.989257)^3 < 2 x^2 y^3 .
\tag{3}
$$

ただし、式 (3) の領域は、図 3 に見て取れるように、

$$
-1.5 < x < 1.5, \qquad
-1 < y < 1.5 ,
\tag{4}
$$

で表される（面積が 7.5 の）長方形に含まれている。[random 関数](https://docs.python.org/ja/3/library/random.html#random.random)をそのまま使うと 0 から 1 までの乱数が返ってくるので、自分で線形変換を施すか、[uniform 関数](https://docs.python.org/ja/3/library/random.html#random.uniform)を用いて、うまく長方形内のランダムな点を発生させること。

![ハート](https://tueda.github.io/PS2020SS/notebooks/images/heart.png)
図3. 式 (3) で表される領域

In [ ]:
# 課題解答9.2  <-- 提出する際に、この行を必ず含めること。

import random




## 発展課題

余裕があればやってみてください。

(3) 四次元空間において、半径が $1$ である四次元球の超体積（つまり中心からの距離が $1$ 以内である点の集合が占める領域）をモンテカルロ法にて求め、結果を誤差とともに表示せよ。ただし、この場合、相対誤差の見積もりを $1 / \sqrt{n}$ と取ると少々甘くなることが考えられるので、代わりに（もう少し正確な）次の式を使って評価せよ[<sup id="cite_ref-6">[6]</sup>](#cite_note-6)。

$$
\text{(相対誤差)}
= \sqrt{\frac{1-r^\star}{n r^\star}} ,
\tag{5}
$$

ここで $r^\star$ は数値実験においてランダムな点が四次元球の内部に入った割合（$=$ `count / n`）である。相対誤差がおおよそ 1% 以下[<sup id="cite_ref-7">[7]</sup>](#cite_note-7)になるようにサンプルサイズを取れ。

In [ ]:
# 課題解答9.3  <-- 提出する際に、この行を必ず含めること。

import random




同じようにして、五次元球、六次元球、...という風に $d$次元球の超体積をモンテカルロ法で計算することを考えることができる。しかし、$d$ が大きくなった時（例えば $d = 20$）、本当にこの方法はうまくいくのだろうか[<sup id="cite_ref-8">[8]</sup>](#cite_note-8)？

## 脚注

<span id="cite_note-1">1.</span> [^](#cite_ref-1)
例えば、（ランダムにするために）目をつぶった状態でダーツをしてみてはどうだろうか？（そもそも正方形の中に入る回数が減りそうなので、非常に効率が悪そうだが。）

<span id="cite_note-2">2.</span> [^](#cite_ref-2)
等号成立（$x^2+y^2=1$）の場合は点が円周上にあることになるが、これを内部に属するとしても、外部に属するとしても、結果には影響しない。実際上、確率が 0 だからである。同様に、正方形内部のランダムな点を発生させるときも、$0 \le x < 1$ の代わりに、$0 \le x \le 1$ としようが、$0 < x < 1$ としようが、同じことである。 

<span id="cite_note-3">3.</span> [^](#cite_ref-3)
円の内部に入るか入らないかが問題なので、（表と裏の出る確率の異なる）コイン投げと同じ要領で議論ができる。いま、確率 $r$（$0 < r < 1$）で 値が $1$、確率 $(1-r)$ で値が $0$ となるような確率変数 $X$ を考えると、期待値と分散は $E[X] = r$、$V[X] = r(1-r)$ である。さて、いまのモンテカルロ法での計算では、これのサンプルサイズ $n$ による標本平均、

$$
\langle X\rangle = \frac{1}{n} \sum_{i=1}^n X_i ,
$$

を取っていることになるから、中心極限定理により、$n$ が十分に大きければこの標本平均 $\langle X\rangle$ は以下のような平均$\mu$、標準偏差$\sigma$の正規分布に従う：

$$
\mu = r , \qquad
\sigma = \sqrt{\frac{r(1-r)}{n}}
= \mu \sqrt{\frac{1-r}{nr}} .
$$

つまり、$1\sigma$に対応する統計誤差（標準誤差（standard error [of the mean])；約 68 %の確率で誤差範囲の中に真の値がある）を許すと、相対誤差は $\sqrt{\frac{1-r}{nr}}$ である。ここで、$r > 1/2$ であれば、 $r$ に依存する因子 $\sqrt{\frac{1-r}{r}}$ は $1$ よりも小さくなる（いまの円周率の数値実験の場合、$r = \pi / 4 > 1 / 2$である）。

<span id="cite_note-4">4.</span> [^](#cite_ref-4)
ただし、（統計誤差が入るモンテカルロ法による数値実験であれ、偶然誤差の入る現実の実験であれ）誤差とはあくまでも測定結果と真の値のずれの程度を統計的に示すもので、その誤差範囲の中に必ず真の値があると主張しているものでは**ない**ことに注意すること。

<span id="cite_note-5">5.</span> [^](#cite_ref-5)
運が悪いと外れる（約 6 %の確率）。

<span id="cite_note-6">6.</span> [^](#cite_ref-6)
$n$は十分に大きいとする。

<span id="cite_note-7">7.</span> [^](#cite_ref-7)
（式 (5) を使うと相対誤差をサンプルサイズだけで決めることができなくなるので）大体で構わない。

<span id="cite_note-8">8.</span> [^](#cite_ref-8)
この問いかけは課題問題 9.3 の一部ではない。が、気になった人のために：キーワード「次元の呪い (curse of dimensionality)」。あと、（直接には関係ない気もするが）「[サクサクメロンパン問題](https://windfall.hatenablog.com/entry/2015/07/02/084623)」で Google 検索するとよいかもしれない。